In [18]:
# defining the website that will be the source of the data
standings_url = "https://fbref.com/en/comps/8/Champions-League-Stats#all_rank_key"

In [19]:
# downloading html of the website
import requests
data = requests.get(standings_url)

In [20]:
# selecting the 'League Table' code
from bs4 import BeautifulSoup
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[8]

IndexError: list index out of range

In [21]:
# finding all 'a' tags and links inside of League Table
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [22]:
# completing the links
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

['https://fbref.com/en/squads/054efa67/Bayern-Munich-Stats',
 'https://fbref.com/en/squads/d48ad4ff/Napoli-Stats',
 'https://fbref.com/en/squads/a77c513e/Benfica-Stats',
 'https://fbref.com/en/squads/53a2f082/Real-Madrid-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/dc56fe14/Milan-Stats',
 'https://fbref.com/en/squads/d609edc0/Internazionale-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/e2d8892c/Paris-Saint-Germain-Stats',
 'https://fbref.com/en/squads/5e876ee6/Porto-Stats',
 'https://fbref.com/en/squads/acbb6a5b/RB-Leipzig-Stats',
 'https://fbref.com/en/squads/add600ae/Dortmund-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/f1e6c5f1/Club-Brugge-Stats',
 'https://fbref.com/en/squads/f0ac8ee6/Eintracht-Frankfurt-Stats',
 'https://fbref.com/en/squads/206d90db/Barcelona-Stats',

In [23]:
# downloading html of one of the teams link - Bayern Munich
data = requests.get(team_urls[0])

In [24]:
# selecting the 'Scores & Fixtures' table code and turning it into pandas data frame
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

ValueError: No tables found

In [8]:
# selecting the link to shooting stats
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [9]:
# completing the link and downloading html
data = requests.get(f"https://fbref.com{links[0]}")

In [10]:
# selecting the link to 'Shooting' website and trurning first table into pandas data frame
shooting = pd.read_html(data.text, match="Shooting")[0]
shooting.head()

For Bayern Munich                                                          \
               Date   Time          Comp         Round  Day Venue Result GF   
0        2022-07-30  20:30  DFL-Supercup  DFL-Supercup  Sat  Away      W  5   
1        2022-08-05  20:30    Bundesliga   Matchweek 1  Fri  Away      W  6   
2        2022-08-14  17:30    Bundesliga   Matchweek 2  Sun  Home      W  2   
3        2022-08-21  17:30    Bundesliga   Matchweek 3  Sun  Away      W  7   
4        2022-08-27  18:30    Bundesliga   Matchweek 4  Sat  Home      D  1   

                      ... Standard               Expected                    \
  GA        Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh G-xG   
0  3      RB Leipzig  ...      NaN  NaN  0     0      NaN  NaN     NaN  NaN   
1  1  Eint Frankfurt  ...     16.6  1.0  0     0      4.0  4.0    0.18  2.0   
2  0       Wolfsburg  ...     18.4  0.0  0     0      2.0  2.0    0.09  0.0   
3  0          Bochum  ...     18.3  1.0  1     1      2.9  2.2    0.11  3.1   
4  1      M'Gladbach  ...     17.4  1.0  0     0      2.8  2.8    0.09 -1.8   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0     NaN        Match Report  
1     2.0        Match Report  
2     0.0        Match Report  
3     2.8        Match Report  
4    -1.8        Match Report  

[5 rows x 26 columns]

In [11]:
# droping one index level
shooting.columns = shooting.columns.droplevel()

In [12]:
# merging data frames using "Date" column as a key
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-07-30,20:30,DFL-Supercup,DFL-Supercup,Sat,Away,W,5.0,3.0,RB Leipzig,...,4-2-3-1,Robert Schröder,Match Report,NaN,12,8,NaN,NaN,0,0
1,2022-08-05,20:30,Bundesliga,Matchweek 1,Fri,Away,W,6.0,1.0,Eint Frankfurt,...,4-4-2,Deniz Aytekin,Match Report,NaN,23,10,16.6,1.0,0,0
2,2022-08-14,17:30,Bundesliga,Matchweek 2,Sun,Home,W,2.0,0.0,Wolfsburg,...,4-2-3-1,Harm Osmers,Match Report,NaN,24,8,18.4,0.0,0,0
3,2022-08-21,17:30,Bundesliga,Matchweek 3,Sun,Away,W,7.0,0.0,Bochum,...,4-4-2,Daniel Siebert,Match Report,NaN,21,11,18.3,1.0,1,1
4,2022-08-27,18:30,Bundesliga,Matchweek 4,Sat,Home,D,1.0,1.0,M'Gladbach,...,4-2-2-2,Daniel Schlager,Match Report,NaN,33,20,17.4,1.0,0,0


In [13]:
# definding seasons thats we will scrape
years = list(range(2023, 2021, -1))
years

[2023, 2022]

In [14]:
all_matches =[]

In [15]:
standings_url = "https://fbref.com/en/comps/8/Champions-League-Stats#all_rank_key"

In [16]:
# scraping mutiple teams for two seasons
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[8]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
         #  team_data = team_data[team_data["Comp"] == "Champions League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

ValueError: No tables found

In [ ]:
len(all_matches)

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

In [ ]:
match_df.to_csv("matches.csv")